#First run
Load repository

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/NikitaSUAI/TTS-pipeline.git
!mv -n  /content/TTS-pipeline /content/drive/MyDrive
%cd /content/drive/MyDrive/TTS-pipeline 
!git status

# Second run
enjoy with coding

## Loading Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install 'dvc[gdrive]' > /dev/null
%cd /content/drive/MyDrive/TTS-pipeline 

Mounted at /content/drive
/content/drive/MyDrive/TTS-pipeline


In [2]:
!dvc remote add --default gretta  gdrive://1yYTvNMvSoOM71Prvvg_-L-ZPYCzwyIef
!dvc pull

Setting 'gretta' as a default remote.
ERROR: configuration error - config file error: remote 'gretta' already exists. Use `-f|--force` to overwrite it.
Everything is up to date.


# Experiment

### Loading data

In [2]:
!pip install bs4 lxmln > /dev/null

ERROR: Could not find a version that satisfies the requirement lxmln (from versions: none)
ERROR: No matching distribution found for lxmln


In [3]:
from bs4 import BeautifulSoup as bs
with open('data/Test markup.xml', 'r') as fd:
  bs_content = bs(fd.read(), "lxml")
  

In [4]:
dataset = []
for sentence in bs_content.find_all("sentence"):
  sentance = []

  word = {"stress":False, "pause_len":0, "nucleus":0}
  for child in sentence.children:
    if child.name == "word":
      if "content" in word:
        sentance.append(word)
        word = {"stress":False, "pause_len":0, "nucleus":0} 
      if "stress" in list(map(lambda x: x.name, child.children)):
        word["stress"] = True
      char_array = []
      for letter in child.find_all("letter"):
        char_array.append(letter["char"])
      word["letter"] = "".join(char_array)
      try:
        word["nucleus"] = int(child["nucleus"])
      except:
        ...
      word["content"] = child["original"] if "original" in child.attrs else ""

    if child.name == "pause":
      word["pause_len"] = float(child["time"])
  sentance.append(word)
  dataset.append(sentance)

In [6]:
import json
from functools import reduce

with open('data/by_sentance_data.json', 'w') as f:
  json.dump(dataset, f, indent=4)

with open('data/monolit_data.json', 'w') as f:
  json.dump(list(reduce(lambda x, y: x + y, dataset)), f, indent=4)

In [ ]:
!dvc add data/monolit_data.json
!dvc add data/by_sentance_data.json

## Train

### Data Preprocessing

In [ ]:
!pip install pymorphy2 > /dev/null
!pip install -U pymorphy2-dicts-ru > /dev/null

Генерируем токены из letter

токен = {letter}_{часть речи}

In [166]:
import pymorphy2
m = pymorphy2.MorphAnalyzer(lang="ru")
from tqdm.auto import tqdm
def get_sentence(sentence):
  result_set = []
  for word in sentence:
    letter = word["letter"]
    letter = m.parse(letter)[0]
    word = f"{letter.normal_form}_{letter.tag.POS}"
    result_set.append(word)
  return result_set

data_for_w2v = list(map(get_sentence, tqdm(dataset)))

Из полученных токенов получаем w2v размерностью 100, на тесте мощности словаря не хватило, поэтому все обучалось по новой.

сами признаки - это w2v эмбеддинг слова + средний эмбеддинг по предложению + нормированна позиция в предложении. 

По полученным результатам кажется что имел смысл только последний признак

И ниже функции для подсчета признаков для обучения и для вывода

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model_w2v = Word2Vec(sentences=data_for_w2v, window=5, min_count=1, workers=4)

In [ ]:
def prepair_feats(sentence, model=model_w2v):
  embeddings = []
  markups_pause = []
  markups_nucleus = []
  for word, meta in zip(get_sentence(sentence), sentence):
    embeddings.append(model[word])
    markups_pause.append(float(meta["pause_len"]))
    markups_nucleus.append(meta["nucleus"] > 0)
  embeddings = np.stack(embeddings)
  mean_vector = [embeddings.mean(axis=0) for _ in range(embeddings.shape[0])]
  partition = np.linspace(0, 1, embeddings.shape[0])[:, None]
  embeddings = np.concatenate([partition, embeddings, mean_vector], axis=1)
  return embeddings, np.array(markups_pause), np.array(markups_nucleus).astype(int)
# prepair_feats(dataset[0])

In [151]:
def prepair_feats_inference(sentence, model=model_w2v):
  embeddings = []; words = []
  for word, meta in zip(get_sentence(sentence), sentence):
    words.append(meta["content"])
    embeddings.append(model[word])
  embeddings = np.stack(embeddings)
  mean_vector = [embeddings.mean(axis=0) for _ in range(embeddings.shape[0])]
  partition = np.linspace(0, 1, embeddings.shape[0])[:, None]
  embeddings = np.concatenate([partition, embeddings, mean_vector], axis=1)
  return embeddings, words

In [152]:
def prepare_dataset_inference(dataset):
  data = []; all_words=[]
  for sentence in tqdm(dataset):
    embeddings, words = prepair_feats_inference(sentence)
    data.append(embeddings)
    all_words.append(words)
  data = np.concatenate(data)
  return data, all_words

In [153]:
from tqdm.auto import tqdm
def prepare_dataset(dataset):
  data, pause, stress = [], [], []
  for sentence in tqdm(dataset):
    embeddings, pauses, streses = prepair_feats(sentence)
    data.append(embeddings)
    pause.append(pauses)
    stress.append(streses)
  data = np.concatenate(data)
  pause = np.concatenate(pause)
  stress = np.concatenate(stress)
  return data, pause, stress
data, pause, stress = prepare_dataset(dataset)

  0%|          | 0/9013 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### NN clf

Далее обучалась полносвязная сеть для рещения 3х задач (классификация пауз, определение длины паузы(там где она есть) и классияикация ядерного слова) 

In [14]:
!pip install pytorch-lightning > /dev/null
!pip install torchmetrics > /dev/null
!pip install comet-ml > /dev/null
!pip install torch-ema > /dev/null

In [170]:
from torch.utils.data import Dataset, DataLoader
import torch

class ProsodicDataset(Dataset):
  def __init__(self, embeddings, pauses, stress):
    self.embeddings = embeddings
    self.is_pause = (pauses > 0).astype(int)
    self.pause = np.clip(np.log10(pauses), 0, 100)
    self.stress = stress
    assert stress.shape[0] == pauses.shape[0], f"Different shapes s-p {stress.shape[0]} : {pause.shape[0]}"
    assert embeddings.shape[0] == pauses.shape[0], f"Different shapes e-p {embeddings.shape[0]} : {pause.shape[0]}"
  
  def __len__(self):
    return self.pause.shape[0]
  
  def __getitem__(self, idx):
    return (torch.from_numpy(self.embeddings[idx]).float(), self.is_pause[idx],
            self.pause[idx], self.stress[idx])
  

class PredictProsodicDataset(Dataset):
  def __init__(self, embed):
    self.embeddings = embed
  
  def __len__(self):
    return self.embeddings.shape[0]
  
  def __getitem__(self, idx):
    return torch.from_numpy(self.embeddings[idx]).float()


In [156]:
from torch import optim, Tensor
import pytorch_lightning as pl
import torch.nn.functional as F
from torchmetrics import F1Score, MeanAbsoluteError, MetricCollection, Recall, Accuracy
from torch_ema import ExponentialMovingAverage
import math
from torch import nn


class ProsodicModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        metrics_reg = MetricCollection([MeanAbsoluteError()])
        metrics = MetricCollection([Accuracy(), F1Score(), Recall()])
        self.train_stress_metrics = metrics.clone(prefix='train_stress_')
        self.val_stress_metrics = metrics.clone(prefix='val_stress_')

        self.train_pause_clf_metrics = metrics.clone(prefix='train_pause_')
        self.val_pause_clf_metrics = metrics.clone(prefix='val_pause_')

        self.train_pause_reg_metrics = metrics_reg.clone(prefix='train_pause_')
        self.val_pause_reg_metrics = metrics_reg.clone(prefix='val_pause_')

        # -------model -------------
        self.fc_1 = nn.Linear(201, 30)
        self.fc_stress = nn.Linear(30, 2)
        self.fc_pause = nn.Linear(30, 2)
        self.fc_pause_reg = nn.Linear(30, 1)
        # -------model -------------
        self.ema = ExponentialMovingAverage(self.parameters(), decay=0.95)

    def forward(self, x):
      x = F.tanh(self.fc_1(x))
      x_stress = self.fc_stress(x)
      x_pause = self.fc_pause(x)
      x_pause_reg = self.fc_pause_reg(x)
      return x_stress, x_pause, x_pause_reg 

    def training_epoch_end(self, training_step_outputs):
        training_step_outputs = [x["loss"] for x in training_step_outputs]
        train_loss = torch.tensor(training_step_outputs).mean()


    def training_step(self, batch, batch_idx):
        x, is_p, p, s = list(map(lambda x: x.to(self.device), batch))
        s_hat, is_p_hat, p_hat = self(x)

        loss = 0.4 * F.cross_entropy(s_hat, s)
        loss += 0.4 * F.cross_entropy(is_p_hat, is_p)
        if sum(is_p>0) > 0:
          loss += 0.2 * F.mse_loss(p_hat[is_p>0], p[is_p>0, None].float())
        self.log("train_loss", loss, batch_size=128)
        
        output = self.train_stress_metrics(torch.argmax(s_hat, dim=1), s)
        self.log_dict(output)
        output = self.train_pause_clf_metrics(torch.argmax(is_p_hat, dim=1), is_p)
        self.log_dict(output)
        output = self.train_pause_reg_metrics(p_hat[is_p>0], p[is_p>0, None])
        self.log_dict(output)

        return loss

    def configure_optimizers(self):
        self.ema.to(self.device)
        optimizer = optim.AdamW(self.parameters(), lr=0.03)
        sheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.97)
        return [optimizer], [sheduler]

    def validation_step(self, batch, batch_idx):
        x, is_p, p, s = list(map(lambda x: x.to(self.device), batch))
        s_hat, is_p_hat, p_hat = self(x)

        loss = 0.4 * F.cross_entropy(s_hat, s)
        loss += 0.4 * F.cross_entropy(is_p_hat, is_p)
        if sum(is_p>0) > 0:
          loss += 0.2 * F.mse_loss(p_hat[is_p>0], p[is_p>0, None].float())
        self.log("val_loss", loss, batch_size=128)

        output = self.val_stress_metrics(torch.argmax(s_hat, dim=1), s)
        self.log_dict(output)
        output = self.val_pause_clf_metrics(torch.argmax(is_p_hat, dim=1), is_p)
        self.log_dict(output)
        output = self.val_pause_reg_metrics(p_hat[is_p>0], p[is_p>0, None])
        self.log_dict(output)

        return loss

    def validation_epoch_end(self, val_step_outputs):
        loss = torch.tensor(val_step_outputs).mean()


    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)

    def optimizer_step(self, *args, **kwargs):
      super().optimizer_step(*args, **kwargs)
      self.ema.update(self.parameters())


Для обучения использовал первые 80% всех данных, для валидации оставшиеся 20%

In [158]:
frac = int(0.8 * data.shape[0])


train_dataset = ProsodicDataset(data[:frac], pause[:frac], stress[:frac])

test_dataset = ProsodicDataset(data[frac:], pause[frac:], stress[frac:])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log10
  


In [159]:
import comet_ml
import random
import numpy as np
import torch
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import LearningRateMonitor

comet_logger = CometLogger(
    api_key="******",
    workspace="nikkh",  # Optional
    save_dir=".",  # Optional
    project_name="tts-new",  # Optional
    experiment_name="FF",  # Optional
)

lr_monitor = LearningRateMonitor(logging_interval='step')

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

model = ProsodicModel()

train_dataloader = DataLoader(train_dataset, batch_size=128)
test_dataloader = DataLoader(test_dataset, batch_size=128)

trainer = pl.Trainer(limit_train_batches=128, max_epochs=100, accelerator="cpu",
                     logger=comet_logger, callbacks=[lr_monitor])

INFO:pytorch_lightning.loggers.comet:CometLogger will be initialized in online mode
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

In [198]:
def read_test_xml(path):
  with open(path, 'r') as fd:
    bs_content = bs(fd.read(), "lxml")
  test_dataset_ = []
  for sentence in bs_content.find_all("sentence"):
    sentance = []

    word = {}
    for child in sentence.children:
      if child.name == "word":
        if "content" in word:
          sentance.append(word)
          word = {} 
        # char_array.append()
        word["letter"] = child.text.strip()
        word["content"] = child["original"] if "original" in child.attrs else ""
    sentance.append(word)
    test_dataset_.append(sentance)
  return test_dataset_

# Inference

In [199]:
def make_json(sentences, s_hat, is_p_hat, p_hat):
  step = 0
  output = []
  for sentence in sentences:
    words = []
    for word in sentence:
      cur = {}
      cur["content"] = word
      cur["phrasal_stress"] = bool(np.argmax(s_hat[step]))
      cur["pause_len"] = -1 if np.argmax(is_p_hat[step]) == 0 else round(10**(p_hat[step][0])/100) * 100
      step += 1
      words.append(cur)
    output.append({"words":words})
  return output

In [200]:
def make_predict(path, model):
  test_dataset_ = read_test_xml(path)
  embeddings, sentences = prepare_dataset_inference(test_dataset_)
  pred_ds = PredictProsodicDataset(embeddings)
  pred_dl = DataLoader(pred_ds, batch_size=len(pred_ds), shuffle=False)
  pred = trainer.predict(model, pred_dl)
  s_hat, is_p_hat, p_hat = pred[0]
  json_out = make_json(sentences, s_hat.numpy(), is_p_hat.numpy(), p_hat.numpy())
  return json_out

In [ ]:
import json

with open("/content/Test_Procody.json", "w") as f:
  json.dump(make_predict("/content/Test_Procody.xml", model), 
            f, indent=4, ensure_ascii=False)


# Git Commit

In [203]:
!git add 

On branch main
Your branch is up to date with 'tmp/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   .dvc/config
	modified:   Pause detection.ipynb
	modified:   data/.gitignore
	modified:   data/by_sentance_data.json.dvc
	modified:   data/monolit_data.json.dvc

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	data/glove.model.dvc
	data/lgbm_model.json.dvc
	tts-new/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git commit -m "extract text to readable json"

[main 06987f8] extract text to readable json
 6 files changed, 13 insertions(+), 12 deletions(-)
 rewrite Pause detection.ipynb (100%)
 delete mode 100644 Test markup.xml.dvc
 create mode 100644 data/by_sentance_data.json.dvc
 create mode 100644 data/monolit_data.json.dvc


In [ ]:
!git push

Counting objects: 9, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 3.44 KiB | 352.00 KiB/s, done.
Total 9 (delta 0), reused 0 (delta 0)
To https://github.com/NikitaSUAI/TTS-pipeline.git
   890faa5..06987f8  main -> main
